In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import csv
import plotly.express as px

In [31]:
development = pd.read_csv(r'C:\Users\feder\Desktop\POLI\DSL\DSL_Winter_Project_2024\DSL_Winter_Project_2024\development.csv')
evaluation = pd.read_csv(r'C:\Users\feder\Desktop\POLI\DSL\DSL_Winter_Project_2024\DSL_Winter_Project_2024\evaluation.csv')

In [32]:
evaluation.isna().any()


Id             False
pmax[0]        False
negpmax[0]     False
area[0]        False
tmax[0]        False
               ...  
pmax[17]       False
negpmax[17]    False
area[17]       False
tmax[17]       False
rms[17]        False
Length: 91, dtype: bool

In [33]:
df = pd.concat([development, evaluation], sort=False)
display(df)

,x,y,pmax[0],negpmax[0],area[0],tmax[0],rms[0],pmax[1],negpmax[1],area[1],...,negpmax[16],area[16],tmax[16],rms[16],pmax[17],negpmax[17],area[17],tmax[17],rms[17],Id
0,200.0,200.0,4.066907,-17.690173,2.847932,0.600000,2.007042,5.409161,-17.721210,4.538778,...,-36.282996,583.899899,72.373094,0.374498,614.916861,-39.848523,591.852768,72.331028,0.405595,NaN
1,200.0,200.0,5.916801,-4.717111,5.792778,79.765174,1.564535,4.414289,-4.736827,3.720435,...,-39.715988,580.042799,71.029155,0.403258,624.950701,-41.266681,586.569646,71.089058,0.405890,NaN
2,200.0,200.0,4.286652,-5.316132,2.356390,74.600000,1.404622,4.567191,-5.985437,3.490490,...,-40.679678,580.407491,71.892264,0.568777,596.437125,-42.712286,574.091695,71.943934,0.498019,NaN
3,200.0,200.0,4.003635,-4.617459,2.189005,43.000000,1.512162,5.019058,-4.229949,6.786200,...,-43.206601,579.882635,72.357388,0.255483,591.763739,-50.681940,584.099483,72.333282,0.336454,NaN
4,200.0,200.0,4.448146,-4.848743,3.997002,196.667482,1.101113,3.250262,-5.783587,2.449456,...,-43.570892,590.156125,71.249130,0.413855,606.917023,-49.923819,584.316142,71.242904,0.293824,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128495,NaN,NaN,5.847174,-3.001292,7.113683,184.584936,1.513353,9.327438,-3.704605,6.657654,...,-41.546677,586.379796,71.379700,0.369140,598.153692,-44.463873,587.177692,71.567752,0.391089,128495.0
128496,NaN,NaN,3.965027,-5.731628,1.695105,121.600000,1.566503,5.833882,-6.185739,4.184656,...,-41.998062,585.524071,71.363987,0.306312,618.483236,-42.177081,583.480972,71.460047,0.342316,128496.0
128497,NaN,NaN,3.630357,-5.045407,2.052289,115.074970,1.493089,19.402353,-8.160696,11.304202,...,-45.808194,585.666913,70.661026,0.458888,592.747090,-49.569852,576.098097,70.806415,0.274790,128497.0
128498,NaN,NaN,3.690116,-5.391879,3.624717,166.925787,1.132331,4.200095,-5.312485,2.652198,...,-43.038669,576.129140,71.226970,0.556171,603.034718,-45.188550,588.892068,71.339138,0.510861,128498.0


The NaN values come from the evaluation dataframe, so we don't have to handle them. The ID column is only present in the evalution dataframe, which is the reason why we find missing values in the df. ID doesn't represent a feature, as its only function is to identify the event to predict. Therefore, we drop this column, together with the outliers column

In [34]:
outliers_idx  = [0,7,12,15,16,17]
dropped_cols = ['pmax[%s]' % i for i in outliers_idx ]+['negpmax[%s]' % i for i in outliers_idx ]+ ['area[%s]' % i for i in outliers_idx ]+ ['tmax[%s]' % i for i in outliers_idx ]+['rms[%s]' % i for i in outliers_idx ]+['Id']
print(dropped_cols)

['pmax[0]', 'pmax[7]', 'pmax[12]', 'pmax[15]', 'pmax[16]', 'pmax[17]', 'negpmax[0]', 'negpmax[7]', 'negpmax[12]', 'negpmax[15]', 'negpmax[16]', 'negpmax[17]', 'area[0]', 'area[7]', 'area[12]', 'area[15]', 'area[16]', 'area[17]', 'tmax[0]', 'tmax[7]', 'tmax[12]', 'tmax[15]', 'tmax[16]', 'tmax[17]', 'rms[0]', 'rms[7]', 'rms[12]', 'rms[15]', 'rms[16]', 'rms[17]', 'Id']


In [39]:
df = df.drop(columns=dropped_cols)

In [40]:
display(df)

,x,y,pmax[1],negpmax[1],area[1],tmax[1],rms[1],pmax[2],negpmax[2],area[2],...,pmax[13],negpmax[13],area[13],tmax[13],rms[13],pmax[14],negpmax[14],area[14],tmax[14],rms[14]
0,200.0,200.0,5.409161,-17.721210,4.538778,22.600000,1.563902,6.084506,-19.892543,3.132328,...,7.454877,-16.861163,4.191909,72.600000,1.184943,6.611877,-17.685799,4.884680,162.800000,1.284969
1,200.0,200.0,4.414289,-4.736827,3.720435,111.875058,1.228058,4.507257,-3.787175,3.267461,...,18.472514,-3.372324,16.620582,119.812512,1.578009,13.802252,-5.154840,8.758870,120.000000,1.093953
2,200.0,200.0,4.567191,-5.985437,3.490490,107.800000,1.015408,5.384155,-3.948853,2.929150,...,7.336668,-5.742383,11.792258,71.908212,1.333036,5.940039,-3.860550,5.340140,6.614830,1.183951
3,200.0,200.0,5.019058,-4.229949,6.786200,175.600000,1.350620,4.165598,-5.576041,1.544370,...,8.450671,-6.342114,4.496741,72.600000,1.945065,6.434910,-5.576315,15.789638,72.195420,1.738998
4,200.0,200.0,3.250262,-5.783587,2.449456,159.800000,0.759515,4.736023,-5.137939,4.211047,...,8.721952,-4.396436,9.127136,71.411535,1.513988,4.483080,-5.361823,3.366542,160.185705,1.323621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128495,NaN,NaN,9.327438,-3.704605,6.657654,71.400000,0.891158,4.763165,-4.856036,11.514778,...,54.351236,-27.172751,26.922277,71.400000,1.085946,22.433310,-9.949075,13.170802,71.200000,1.747292
128496,NaN,NaN,5.833882,-6.185739,4.184656,20.037256,1.422575,3.790421,-5.154848,4.475711,...,15.090143,-6.553778,8.782450,71.600000,1.425143,4.467065,-4.729529,2.264683,84.400000,1.617628
128497,NaN,NaN,19.402353,-8.160696,11.304202,70.800000,1.370319,17.555460,-8.818259,9.256798,...,23.518393,-9.069559,13.958058,70.800000,1.625532,4.901157,-4.210786,2.049936,66.800000,0.908640
128498,NaN,NaN,4.200095,-5.312485,2.652198,71.400000,0.932871,2.959903,-4.739268,0.946364,...,14.752425,-5.178433,9.768462,71.425166,2.192589,11.362302,-6.197583,5.001778,94.800000,0.739196


The samples with the non-missing x and y values belong to the training set, the rest will belong to the test set

In [22]:
train_mask = ~df1[['x','y']].isna()
train_mask

,x,y
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
128495,False,False
128496,False,False
128497,False,False
128498,False,False


In [46]:
coord = ['x', 'y']
test_mask= df['x'].isna() & df['y'].isna()

X = df.drop(columns=coord).values
y = df[coord].values

X_train_valid=X[~test_mask]
y_train_valid=y[~test_mask]

X_test=X[test_mask]
y_test=y[~test_mask]

from sklearn.model_selection import train_test_split
rs=2024 #we put random state so that whenever we run the cells, the outputs remain the same
X_train, X_valid, y_train,y_valid= train_test_split(X_train_valid,y_train_valid,shuffle=True, random_state=rs)

fr

In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

forest = RandomForestRegressor(random_state=rs)
forest.fit(X_train, y_train)
r2_score(y_valid, forest.predict(X_valid))

0.9988553809611789

In [51]:
features = df[~test_mask].drop(columns=coord).columns

sorted(zip(features, forest.feature_importances_), key=lambda x: x[1], reverse=True)

[('pmax[8]', 0.35806500077877096),
 ('pmax[11]', 0.23897217123870848),
 ('pmax[10]', 0.1611268085403518),
 ('pmax[5]', 0.07527291837424653),
 ('pmax[13]', 0.03660228113127059),
 ('pmax[9]', 0.03118804563463314),
 ('negpmax[13]', 0.026669124771999748),
 ('negpmax[11]', 0.012926494604937125),
 ('negpmax[3]', 0.010385886130506438),
 ('negpmax[10]', 0.00766529654040441),
 ('pmax[4]', 0.007302987815857915),
 ('negpmax[8]', 0.00535208263496631),
 ('pmax[3]', 0.004613734498244714),
 ('negpmax[1]', 0.004440152155950458),
 ('negpmax[5]', 0.004223524184754046),
 ('negpmax[6]', 0.003152299776109825),
 ('negpmax[14]', 0.0020305656993489284),
 ('negpmax[4]', 0.00199127087204711),
 ('pmax[2]', 0.0019313507669337477),
 ('negpmax[9]', 0.0012928066166994665),
 ('pmax[1]', 0.0012742259982067132),
 ('pmax[6]', 0.0010996079603410197),
 ('pmax[14]', 0.0010183971864801261),
 ('area[3]', 0.00024094004472058358),
 ('negpmax[2]', 0.00017693208524918638),
 ('area[5]', 0.00016413207850566488),
 ('area[13]', 0.00

We see that both rms and tmax features have an importance score close to zero. However our r2 score is already of 0.99885530, so removing rms and tmax's columns will provide little improvement of the r2 score